In [1]:
from pathlib import Path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
CRED_PATH = Path('/home/benjamin/predicatestudio/ps-sandbox/IndecorOdoo/email_bot/IndecorEmailBot.json')
TOKEN_PATH = Path("token.json")

class OAuthGmailUser():
    def __init__(self, cred_path="credentials.json", token_path="token.json", scopes=['send']):
        # using Path objects to hold credential paths
        self.cred_path = Path(cred_path)
        self.token_path = Path(token_path)
        self.scopes = [f"https://www.googleapis.com/auth/gmail.{scope}" for scope in scopes]
        # full scope encapsulates all other scopes
        if "full" in self.scopes:
            self.scopes = ['https://mail.google.com/']
        
        self.creds = self._get_creds()
    
    def _read_token(self):
        if self.token_path.exists():
            return Credentials.from_authorized_user_file(self.token_path.absolute(), self.scopes)
        

    def _write_token(self, creds):
        with self.token_path.open('w') as token_file:
            token_file.write(creds.to_json())

    def _refresh_token(self, creds):
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
            self._write_token(creds)
        else:
            # discard expired creds that won't refresh
            creds = None
        return creds

    def _generate_creds(self):
        flow = InstalledAppFlow.from_client_secrets_file(
            self.cred_path, self.scopes)
        creds = flow.run_local_server(port=0)
        self._write_token(creds)
        return creds

    
    def _get_creds(self):
        # If we have user access and refresh tokens, use those. Otherwise, generate 
        # them from the OAuth credentials
        creds = self._read_token()
                
        # If there are no (valid) credentials available, let the user log in.
        if creds and not creds.valid:
            creds = self._refresh_token(creds)
        if not creds:
            creds = self._generate_creds()
        return creds
    
    def start_service(self):
        service = build('gmail', 'v1', credentials=self.creds)
        return service

def read_email(user):
    pass

# def main():
#     """Shows basic usage of the Gmail API.
#     Lists the user's Gmail labels.
#     """
    
#     creds = get_user_token(TOKEN_PATH, SCOPES)


#     service = build('gmail', 'v1', credentials=creds)

#     # Call the Gmail API
#     results = service.users().labels().list(userId='me').execute()
#     labels = results.get('labels', [])

#     if not labels:
#         print('No labels found.')
#     else:
#         print('Labels:')
#         for label in labels:
#             print(label['name'])

# if __name__ == '__main__':
#     main()

In [3]:
Indecor = OAuthGmailUser(CRED_PATH.absolute(), TOKEN_PATH.absolute(), ["modify"])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=467488332884-nkn4j4q35qgq66vterij95d56limmhn5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58565%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=czOT4mMAxIugsjp9QTSm57XPYpgqjd&access_type=offline


In [4]:
service = Indecor.start_service()

In [7]:
users = service.users()

In [11]:
threads = users.threads()

In [ ]:
threads.list()